In [1]:
#!pip install --target=$my_path catboost

In [2]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split,cross_validate, KFold, StratifiedKFold, GridSearchCV # 학습, 테스트set 구분
import sklearn.metrics as mt # 성능지표를 계산하기 위해 import
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

import warnings
warnings.filterwarnings('ignore')

In [3]:
df  = pd.read_csv('/content/drive/MyDrive/Dacon/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/Dacon/test.csv')

In [4]:
df

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,...,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [5]:
df.columns

Index(['index', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10',
       'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20',
       'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'introelapse',
       'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [6]:
# train = df.drop(['index', 'country','Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26'],axis = 1)
# test =test.drop(['index', 'country','Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26'],axis = 1)

In [7]:
train = df.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [8]:
#train을 target과 feature로 나눠줍니다.
train_x=train.drop(['nerdiness'], axis=1)
train_y=train['nerdiness']

In [9]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=seed, stratify = train_y)

In [10]:
ml_lgbm = LGBMClassifier(
            n_estimators=3000
        )

In [11]:
# ml_lgbm = LGBMClassifier(
#             n_estimators=1000,
#             learning_rate=0.02,
#             random_state=94,
#             max_depth=11,
#             num_leaves=573,
#             min_child_samples=10,
#             subsample=0.76,
#             colsample_bytree=0.52,
#             max_bin=449,
#             reg_lambda=0.07,
#             reg_alpha=0.28
#         )

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

kfold = StratifiedKFold(n_splits=5)
cv_accuracy = []
n_iter =0

for train_index, test_index in kfold.split(train_x, train_y):  # feautres 데이터를 위에서 지정한 kfold 숫자로 분할
    x_valid, x_test_fold = train_x.iloc[train_index], train_x.iloc[test_index]
    y_valid, y_test_fold = train_y[train_index], train_y[test_index]
    
    ml_lgbm.fit(x_valid, y_valid)
    pred = ml_lgbm .predict(x_test_fold)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test_fold, pred), 4) # 소수점 4자리 반올림
    train_size = x_valid.shape[0]
    test_size = x_test_fold.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 0.787,  학습 데이터 크기 : 12000,  검증 데이터 크기 : 3000
#1 검증 세트 인덱스 : [   0    1    2 ... 3027 3030 3031]

#2 교차 검증 정확도 : 0.7957,  학습 데이터 크기 : 12000,  검증 데이터 크기 : 3000
#2 검증 세트 인덱스 : [2962 2966 2968 ... 6026 6028 6029]

#3 교차 검증 정확도 : 0.7823,  학습 데이터 크기 : 12000,  검증 데이터 크기 : 3000
#3 검증 세트 인덱스 : [5966 5967 5969 ... 9034 9035 9040]

#4 교차 검증 정확도 : 0.7947,  학습 데이터 크기 : 12000,  검증 데이터 크기 : 3000
#4 검증 세트 인덱스 : [ 8976  8977  8978 ... 12037 12038 12041]

#5 교차 검증 정확도 : 0.7733,  학습 데이터 크기 : 12000,  검증 데이터 크기 : 3000
#5 검증 세트 인덱스 : [11961 11962 11964 ... 14997 14998 14999]

## 평균 검증 정확도: 0.7866


In [13]:
# ml_lgbm.fit(x_train, y_train)

clf_pred = ml_lgbm.predict(x_test)
reg_pred = ml_lgbm.predict_proba(x_test)[:,1] #proba를 통해 확률값으로 변환
print(roc_auc_score(y_test, clf_pred))
print(roc_auc_score(y_test, reg_pred))

0.9535626413375858
0.9909126314163542


In [14]:
# ml_lgbm.fit(x_train, y_train)

# ml_lgbm_pred = ml_lgbm.predict(x_test)

In [15]:
# # 학습결과 평가
# print("Train_Accuracy : ", ml_lgbm.score(x_train, y_train))
# print("Test_Accuracy : ", ml_lgbm.score(x_test, y_test), '\n')

# accuracy = mt.accuracy_score(y_test, ml_lgbm_pred)
# print("Accuracy :" ,mt.accuracy_score(y_test, ml_lgbm_pred))

# Submission  
모델이 split되지 않은 전체 train에 대해 학습하고 test 데이터를 이용하여 predict 수행  

In [16]:
lgbm_pred = ml_lgbm.predict_proba(test)[:,1]

In [17]:
# # submission용. train 전체 학습 & test에 대해 predict 진행
# lgbm_clf = LGBMClassifier(
#             n_estimators=1000
#         )
# lgbm_clf.fit(train_x, train_y)

# lgbm_pred = lgbm_clf.predict(test)

In [18]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [19]:
submission["nerdiness"] = lgbm_pred

In [20]:
submission

,index,nerdiness
0,0,0.000258
1,1,0.994486
2,2,0.994647
3,3,0.978564
4,4,0.881825
...,...,...
35447,35447,0.999889
35448,35448,0.995488
35449,35449,0.999992
35450,35450,0.000094


In [21]:
submission.to_csv("/content/drive/MyDrive/Dacon/LGBM.csv", index = False)